In [134]:
import os
import sys
from pathlib import Path
import pandas as pd
from datetime import datetime
#import sqlalchemy as sa

In [273]:
sys.path.append(r"../../src/collectcube")
from db import *
from db_post import *

In [136]:
#out_dir = '../../data/LUCinLA_paraguay/smSamp'
#local_db_path = os.path.join(out_dir, 'landcover.db')
out_dir = 'C:/GISprojects/ParaguayValidation/smDistricts'
local_db_path = 'C:/GISprojects/ParaguayValidation/smDistricts/landcover.db'

In [274]:
table_check = check_table(local_db_path,'pixels')

             PID  Center     cent_X     cent_Y   cent_lat  cent_long  ransamp  \
49495  0005499_4       0  2974500.0 -3213020.0 -25.431507 -57.444894        1   
49496  0005499_5       0  2974520.0 -3213020.0 -25.431507 -57.444675        1   
49497  0005499_6       0  2974500.0 -3213030.0 -25.431589 -57.444883        1   
49498  0005499_7       0  2974510.0 -3213030.0 -25.431589 -57.444774        1   
49499  0005499_8       0  2974520.0 -3213030.0 -25.431589 -57.444665        1   

       checked  PID0  PID1          sampgroup  
49495        0  5499     4  D1119_Central_JAS  
49496        0  5499     5  D1119_Central_JAS  
49497        0  5499     6  D1119_Central_JAS  
49498        0  5499     7  D1119_Central_JAS  
49499        0  5499     8  D1119_Central_JAS  


## Data cleaning (e.g. if pandas import below doesn't work)
Note: need for this should be eliminated as more formatting restrictions are placed in db entry functions

### manipulating table (should not need to do this here)

In [ ]:
'''
con = sqlite3.connect(local_db_path)
c = con.cursor()
## adding a new column to existing table
c.execute("ALTER TABLE PixelVerification ADD COLUMN entry_lev INTEGER")
## filling column in existing table
#c.execute("UPDATE PixelVerification SET entry_lev = 4 WHERE entry_lev IS NULL")
con.commit()      
c.close()

### querying table (e.g. for bad records)

In [ ]:
'''
con = sqlite3.connect(local_db_path)
c = con.cursor()
c.execute("SELECT * FROM PixelVerification  WHERE imgDate = '2021_03-07'")
con.commit()
print(c.fetchone()) 
'''

con = sqlite3.connect(local_db_path)
c = con.cursor()
c.execute("SELECT * FROM pixels WHERE PID0 = '500'")
con.commit()
print(c.fetchone()) 

### fixing bad records

In [266]:
'''
con = sqlite3.connect(local_db_path)
c = con.cursor()
c.execute("UPDATE PixelVerification SET imgDate = '2023-04-21' WHERE imgDate ='2023--04-21'")
con.commit()
c.close()
'''


con = sqlite3.connect(local_db_path)
c = con.cursor()
c.execute("UPDATE pixels SET sampgroup = 'D0112_Concepcion_A' WHERE PID0 ='998'")
con.commit()
c.close()


'''
con = sqlite3.connect(local_db_path)
c = con.cursor()
c.execute("UPDATE PixelVerification SET PID0 = '4312' WHERE recID = '8588'")
con.commit()
c.close()
'''

'\ncon = sqlite3.connect(local_db_path)\nc = con.cursor()\nc.execute("UPDATE PixelVerification SET PID0 = \'4312\' WHERE recID = \'8588\'")\ncon.commit()\nc.close()\n'

## Open db table as pandas dataframe

In [139]:
## This uses sqlalchemy but is prefered because it will make sure that formatting is correct
df = open_data_records_sqlachemy(local_db_path)
df.tail()

,recID,PID0,PID1,imgDate,LC5,LC,HOMONBHD9,ForestProx,WaterProx,PercentTree,...,TREEPLANT0,HIGHVEG,TREEPLANT,FOREST,Age,Stability,State,Notes,entry_lev,PIDi
10622,12546,1982,0,2022-01-10,70.0,80.0,5.0,,,60,...,,,,100,,--,--,recent disturbance,0,0001982_0
10623,12547,1982,0,2021-10-11,70.0,80.0,5.0,30,,70,...,,,,100,,--,--,,0,0001982_0
10624,12548,1982,0,2006-07-05,70.0,80.0,5.0,50,,70,...,,,,100,,--,--,,0,0001982_0
10625,12549,1982,0,2011-08-15,70.0,80.0,5.0,50,,70,...,,,,100,,--,--,,0,0001982_0
10626,12550,1982,0,2015-04-21,70.0,80.0,5.0,50,,70,...,,,,100,,--,--,,0,0001982_0


In [6]:
## This uses only sqllite and can be used to examine table if the above fails
conn = sqlite3.connect(local_db_path, isolation_level=None,
    detect_types=sqlite3.PARSE_COLNAMES)
db_df = pd.read_sql_query("SELECT * FROM PixelVerification", conn)
db_df.tail()
#db_df.to_csv(os.path.join('C:/GISprojects/ParaguayValidation/smDistricts','sm_test_db.csv'), index=False)

,recID,PID,PID0,PID1,imgDate,LC5,LC,HOMONBHD9,ForestProx,WaterProx,...,MEDVEG,TREEPLANT0,HIGHVEG,TREEPLANT,FOREST,Age,Stability,State,Notes,entry_lev
7259,9129,4051_0,4051,0,2021-07-07,20,31,8.0,,,...,,,,,,,--,Mature,,4
7260,9130,4051_0,4051,0,2018-12-22,20,31,8.0,,,...,,,,,,,--,Mature,,4
7261,9131,4051_0,4051,0,2017-07-25,20,13,8.0,,,...,,,,,,,--,Mature,,4
7262,9132,4051_0,4051,0,2013-11-04,20,13,8.0,,,...,,,,,,,--,Mature,,4
7263,9133,4051_0,4051,0,2007-07-28,20,13,8.0,,,...,,,,,,,--,Mature,,4


In [7]:
#addfile = 'C:/GISprojects/ParaguayValidation/smDistricts/sm_add.csv'
#pd.read_csv(addfile).to_sql('PixelVerification', conn, if_exists='append', index=False)

3208

## Get sample for target date (direct and indirect)

In [269]:
## Will consider anything 1-June of prior year to 1-June of target_date year as direct observation 
## and interpolate from other observations according to the rules set up

target_date = 2022

sample_type = 'area_est' ## 'area_est',  validation', 'training' 
sample_cats = 'bi_crop'   ## 'bi_tree', 'bi_forest'
lut = '../../data/Class_LUT.csv'
   
all_obs=get_sample_for_date(target_date,local_db_path,sample_type,sample_cats,lut)
    
all_obs.sort_values('PID0')
all_obs.head(n=10)
#all_obs.to_csv(os.path.join(out_dir,'samp2022.csv'))

there are 4646 observations for selected date
there are 4429 center pixel observations


,recID,PID0,PID1,imgDate,LC5,LC,HOMONBHD9,ForestProx,WaterProx,PercentTree,...,HIGHVEG,TREEPLANT,FOREST,Age,Stability,State,Notes,entry_lev,PIDi,obs_type
624,1.0,0,0,2022-11-25,99.0,98.0,8.0,,,,...,,,,,--,--,,4,0000000_0,indirect_GE
308,1997.0,1,0,2021-07-07,99.0,98.0,8.0,,,,...,,,,,--,--,,1,0000001_0,indirect_GE
1778,1999.0,2,0,2021-07-06,20.0,30.0,NaN,None,None,None,...,None,None,None,None,None,None,,1,0000002_0,direct_GE
1267,2001.0,3,0,2022-07-01,70.0,80.0,7.0,,,,...,,,,,--,--,,1,0000003_0,indirect_GE
310,2003.0,4,0,2022-07-01,-1.0,30.0,NaN,None,None,None,...,None,None,None,None,None,None,,1,0000004_0,indirect_GE
1787,2008.0,5,0,2021-07-06,20.0,30.0,NaN,None,None,None,...,None,None,None,None,None,None,,1,0000005_0,direct_GE
1268,2010.0,6,0,2023-12-01,99.0,98.0,NaN,None,None,None,...,None,None,None,None,None,None,,1,0000006_0,indirect_GE
1269,2011.0,7,0,2023-06-17,99.0,98.0,NaN,None,None,None,...,None,None,None,None,None,None,,1,0000007_0,indirect_GE
312,2013.0,8,0,2022-01-07,99.0,98.0,NaN,None,None,None,...,None,None,None,None,None,None,,1,0000008_0,indirect_GE
1270,2014.0,9,0,2023-12-21,70.0,80.0,NaN,None,None,None,...,None,None,None,None,None,None,,1,0000009_0,indirect_GE


## Create point file for confusion matrices

In [142]:
pix = open_pix_info_sqlachemy(local_db_path)

pix_data = pd.merge(all_obs,pix,left_on='PIDi',right_on='PID',how='left')
pix_data.head()
pix_data.to_csv(os.path.join(out_dir,'samp2022_pix.csv'))

## Summarize sample size by sample group

In [272]:
pix = open_pix_info_sqlachemy(local_db_path)
pix_data = pix_data[pix_data['PID1']==0]
pix_data = pd.merge(all_obs,pix,left_on='PIDi',right_on='PID',how='left')
samp_sum=pix_data.groupby(['sampgroup','LC']).size().reset_index(name='counts')
samp_tot=pix_data.groupby(['sampgroup']).size().reset_index(name='total')
samp = pd.merge(samp_sum,samp_tot,on='sampgroup',how='left')
samp['per'] = samp['counts']/samp['total']
samp[samp['LC']==30.0]

,sampgroup,LC,counts,total,per
0,D0102_Concepcion_B,30.0,71,499,0.142285
3,D0103_Concepcion_H,30.0,28,243,0.115226
6,D0112_Concepcion_A,30.0,16,181,0.088398
9,D0217_SanPedro_C,30.0,120,489,0.245399
12,D0301_Cordillera_C,30.0,23,413,0.055690
15,D0502_Caaguazu_C,30.0,107,472,0.226695
18,D0518_Caaguazu_3F,30.0,169,508,0.332677
21,D0603_Caazapa_B,30.0,58,406,0.142857
24,D0607_Caazapa_SJN,30.0,44,340,0.129412
27,D0806_Misiones_SM,30.0,70,338,0.207101
